In [1]:
import pandas as pd

In [2]:
import fasttext

In [3]:
def load_csv(path):
    df = pd.read_csv(path)
    df = df.assign(xdelta=df.x0.diff(),
                   ydelta=df.doctop.diff(),
                   height=df.bottom-df.top,
                   width=df.x1-df.x0) 
    df = df.assign(xdd=df.xdelta.diff(),
                   ydd=df.ydelta.diff())
    df = df.fillna(0)
    return df

In [4]:
df = pd.concat([load_csv(f)
                for f in (
                "../data/train/Rgl-1314-2021-L-Lotissement.csv",
                "../data/train/00-Reglement-1000-2008-PPC.csv",
                "../data/train/Rgl-1314-2021-PC-version-en-vigueur-20230509.csv",
                "../data/train/Rgl-1314-2021-TM-Travaux-municipaux.csv",
                "../data/train/2022-03-21-Reglement-PAE-Adopte.csv",
                "../data/train/2022-04-19-Rgl-1324-redevances-adopte.csv",
                "../data/train/xx-2020-04-20-RGL-1289-Formation-CCE-adopte_1.csv",
                )], ignore_index=True)

In [5]:
import numpy as np

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-06-27 09:56:13.419219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
from sklearn.preprocessing import StandardScaler
import re
indexlike = re.compile(r"^(\d+|[a-z])[\)\.]|-$")
scaler = StandardScaler()
scaler.fit(df.loc[:, ["x0", "top", "width", "height"]])
ft = fasttext.load_model("../alexi/extraction/rnn99.fasttext")

def make_features(df):
    feats = df.loc[:,["xdelta", "ydelta", "x0", "top", "xdd", "ydd", "width", "height"]]
    index = []
    idelta = []
    isnum = []
    iseq = []
    chapitre = []
    section = []
    prevnum = 0
    for t in df.loc[:, "text"]:
        t = str(t)  # fuck you, pandas
        isnum.append(t.isnumeric())
        chapitre.append("chap" in t.lower())
        section.append("sect" in t.lower())
        if m := indexlike.match(t):
            index.append(True)
            if m.group(1) and m.group(1).isnumeric():
                idx = int(m.group(1))
                idelta.append(idx - prevnum)
                iseq.append(idx == prevnum + 1)
                prevnum = idx
            else:
                idelta.append(0)
                iseq.append(False)
        else:
            index.append(False)
            idelta.append(0)
            iseq.append(False)
    feats = feats.assign(# Seems not to help
                         # xdelta=np.sign(df.loc[:, "xdelta"]),
                         # ydelta=np.sign(df.loc[:, "ydelta"]),
                         # Does seem useful (et pour cause)
                         xdd=np.sign(df.loc[:, "xdd"]),
                         ydd=np.sign(df.loc[:, "ydd"]))
    scaled = scaler.transform(df.loc[:, ["x0", "top", "width", "height"]])
    feats = feats.assign(x0=scaled[:,0], top=scaled[:, 1], width=scaled[:,2], height=scaled[:,3])
    feats = feats.assign(index=index, idelta=idelta,
                         isnum=isnum, iseq=iseq,
                         chapitre=chapitre, section=section)
    return feats
def make_targets(df):
    return df["tag"]
def make_words(df):
    return np.array([ft.get_word_vector(w) for w in df["text"]])


features = make_features(df)
targets = make_targets(df)
words = make_words(df)
features.head()

,xdelta,ydelta,x0,top,xdd,ydd,width,height,index,idelta,isnum,iseq,chapitre,section
0,0.00000,0.00000,-1.603838,0.294861,0.0,0.0,3.854404,22.546310,False,0,False,False,False,False
1,123.01296,0.00000,-0.587435,0.294861,0.0,0.0,0.185036,22.546310,False,0,False,False,False,False
2,33.61764,0.00000,-0.309666,0.294861,-1.0,0.0,4.213874,22.546310,False,0,False,False,False,False
3,-156.63060,28.44948,-1.603838,0.415270,-1.0,1.0,0.711375,2.729439,False,0,False,False,False,False
4,45.47600,0.00000,-1.228089,0.415270,1.0,-1.0,1.737252,2.729439,False,0,False,False,False,False


In [8]:
import pickle

In [9]:
with open("../alexi/extraction/rnn99.scaler.pkl", "wb") as outfh:
    pickle.dump(scaler, outfh)

In [10]:
features.shape, targets.shape

((52166, 14), (52166,))

In [11]:
lookup = tf.keras.layers.StringLookup()
lookup.adapt(targets)
targets = lookup(targets)

2023-06-27 09:56:17.536822: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-27 09:56:17.567072: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-27 09:56:17.567350: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-27 09:56:17.568319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild Te

In [12]:
import json
with open("../alexi/extraction/rnn99.vocab.json", "w") as outfh:
    json.dump(lookup.get_vocabulary(), outfh)

In [68]:
BLOCKSIZE=128
def make_blocks(seq, dtype="int32"):
    blocks = [seq[start:start+BLOCKSIZE] for start in range(0, seq.shape[0], BLOCKSIZE)]
    return keras.utils.pad_sequences(blocks, padding="post", dtype=dtype)

xf_train_blocks = make_blocks(features, "float32")
xw_train_blocks = make_blocks(words, "float32")
y_train_blocks = make_blocks(targets)

develdf = load_csv("../data/train/Rgl-1314-2021-DM-Derogations-mineures.csv")
xf_devel_blocks = make_blocks(make_features(develdf), "float32")
xw_devel_blocks = make_blocks(make_words(develdf), "float32")
y_devel_blocks = make_blocks(lookup(make_targets(develdf)))

In [69]:
xf_train_blocks.shape, xw_train_blocks.shape, y_train_blocks.shape, xf_devel_blocks.shape, y_devel_blocks.shape

((408, 128, 14), (408, 128, 16), (408, 128), (26, 128, 14), (26, 128))

In [108]:
input_features = layers.Input(name="features", shape=(BLOCKSIZE, features.shape[1]))
input_words = layers.Input(name="words", shape=(BLOCKSIZE, words.shape[1]))
concat = layers.concatenate([input_features, input_words])
mask = layers.Masking()(concat)
lstm = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(mask)
outputs = layers.Dense(lookup.vocabulary_size(), activation="softmax", name="predictions")(lstm)
model = keras.Model(inputs=[input_features, input_words], outputs=outputs)

In [109]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=500,
    decay_rate=0.9)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [110]:
callback = tf.keras.callbacks.EarlyStopping(start_from_epoch=50,
                                            monitor='val_sparse_categorical_accuracy',
                                            patience=20, restore_best_weights=True)
model.fit((xf_train_blocks, xw_train_blocks), y_train_blocks,
          batch_size=16, epochs=1000, # sample_weight=y_train_weights,
          validation_data=((xf_devel_blocks, xw_devel_blocks), y_devel_blocks),
          callbacks=(callback,))

Epoch 1/1000
26/26 [==============================] - 11s 129ms/step - loss: 1.8849 - sparse_categorical_accuracy: 0.3806 - val_loss: 1.2932 - val_sparse_categorical_accuracy: 0.4475
Epoch 2/1000
26/26 [==============================] - 1s 26ms/step - loss: 1.2233 - sparse_categorical_accuracy: 0.5036 - val_loss: 1.0861 - val_sparse_categorical_accuracy: 0.5609
Epoch 3/1000
26/26 [==============================] - 1s 25ms/step - loss: 1.0917 - sparse_categorical_accuracy: 0.5675 - val_loss: 0.8984 - val_sparse_categorical_accuracy: 0.6403
Epoch 4/1000
26/26 [==============================] - 1s 25ms/step - loss: 0.8939 - sparse_categorical_accuracy: 0.6780 - val_loss: 0.8177 - val_sparse_categorical_accuracy: 0.6440
Epoch 5/1000
26/26 [==============================] - 1s 28ms/step - loss: 0.7496 - sparse_categorical_accuracy: 0.7278 - val_loss: 0.5091 - val_sparse_categorical_accuracy: 0.8609
Epoch 6/1000
26/26 [==============================] - 1s 24ms/step - loss: 0.5396 - sparse_ca

26/26 [==============================] - 1s 24ms/step - loss: 0.0358 - sparse_categorical_accuracy: 0.9921 - val_loss: 0.0408 - val_sparse_categorical_accuracy: 0.9879
Epoch 47/1000
26/26 [==============================] - 1s 26ms/step - loss: 0.0333 - sparse_categorical_accuracy: 0.9930 - val_loss: 0.0408 - val_sparse_categorical_accuracy: 0.9879
Epoch 48/1000
26/26 [==============================] - 1s 25ms/step - loss: 0.0311 - sparse_categorical_accuracy: 0.9935 - val_loss: 0.0383 - val_sparse_categorical_accuracy: 0.9889
Epoch 49/1000
26/26 [==============================] - 1s 25ms/step - loss: 0.0297 - sparse_categorical_accuracy: 0.9938 - val_loss: 0.0386 - val_sparse_categorical_accuracy: 0.9895
Epoch 50/1000
26/26 [==============================] - 1s 25ms/step - loss: 0.0286 - sparse_categorical_accuracy: 0.9942 - val_loss: 0.0385 - val_sparse_categorical_accuracy: 0.9886
Epoch 51/1000
26/26 [==============================] - 1s 25ms/step - loss: 0.0275 - sparse_categorical_

In [111]:
model.evaluate((xf_train_blocks, xw_train_blocks), y_train_blocks)

13/13 [==============================] - 0s 17ms/step - loss: 0.0407 - sparse_categorical_accuracy: 0.9893


[0.04069480672478676, 0.9893225431442261]

In [112]:
model.evaluate((xf_devel_blocks, xw_devel_blocks), y_devel_blocks)

1/1 [==============================] - 0s 44ms/step - loss: 0.0306 - sparse_categorical_accuracy: 0.9926


[0.030638784170150757, 0.9925834536552429]

In [113]:
def chunk(df, model):
    xf_blocks = make_blocks(make_features(df), "float32")
    xw_blocks = make_blocks(make_words(df), "float32")
    vocab = lookup.get_vocabulary()
    predictions = [vocab[i] for i in np.concatenate(model.predict((xf_blocks, xw_blocks)).argmax(axis=2))]
    words = df.loc[:, "text"]
    sentences = []
    chunk = []
    tag = None
    for label, word in zip(predictions, words):
        if label[0] == 'B':
            if chunk:
                sentences.append((tag, " ".join(str(x) for x in chunk)))
                chunk = []
            tag = label.partition('-')[2]
        elif len(chunk) == 0:
            tag = label.partition('-')[2]
        chunk.append(word)
    sentences.append((tag, " ".join(str(x) for x in chunk)))
    return sentences

In [114]:
def tag(df, model):
    xf_blocks = make_blocks(make_features(df), "float32")
    xw_blocks = make_blocks(make_words(df), "float32")
    vocab = lookup.get_vocabulary()
    predictions=[vocab[i] for i in np.concatenate(model.predict((xf_blocks, xw_blocks)).argmax(axis=2))]
    return predictions[:df.shape[0]]

In [115]:
model.save("../alexi/extraction/rnn99")

INFO:tensorflow:Assets written to: ../alexi/extraction/rnn99/assets


INFO:tensorflow:Assets written to: ../alexi/extraction/rnn99/assets


In [116]:
chunk(load_csv("../data/1140-2010-Creation-du-CCE.csv"), model)

1/1 [==============================] - 2s 2s/step


[('',
  'VILLE DE SAINTE-ADÈLE PROVINCE DE QUÉBEC COMTE DE BERTRAND RÈGLEMENT NO. 1140-2010 A une séance ordinaire du Conseil municipal, tenue publiquement le 19 juillet 2010 à 20h, dans la salle du Conseil Municipal située au 1386 de la rue Dumouchel, Sainte- Adèle, province de Québec, lieu ordinaire des séances à laquelle étaient présents Mesdames et Messieurs les Conseillers Nadine Brière District 1 Lise Gendron District 3 John Butler District 4 Robert Lagacé District 5 Pierre Morabito District 6'),
 ('Alinea', 'sous la présidence de Monsieur le Maire Réjean Charbonneau.'),
 ('Alinea',
  "Tous membres dudit Conseil et en formant le quorum. Conformément aux dispositions de l’article 356 de la LOI SUR LES CITÉS ET VILLES (L.R.Q. 1977, ch. C-19), le greffier est dispensé de la lecture du règlement. Règlement numéro 1140-2010 décrétant la formation d’un Comité Consultatif en Environnement sur le territoire de la Ville de Sainte-Adèle. ATTENDU QUE l'avis de motion du présent règlement a 

In [117]:
chunk(load_csv("../data/00-Reglement-1233-Comite-du-patrimoine-VF.csv"), model)

1/1 [==============================] - 0s 28ms/step


[('',
  'RÈGLEMENT NUMÉRO 1233 concernant la formation d’un comité du patrimoine Séance ordinaire du conseil municipal, tenue publiquement le 15 août 2016 à 20h, dans la salle du conseil municipal située au 1386, rue Dumouchel, Sainte-Adèle, lieu ordinaire des séances à laquelle étaient présents mesdames et messieurs les conseillers Nadine Brière District 1 Roch Bédard District 2 Gilles Legault District 3 John Butler District 4 Diane de Passillé District 6'),
 ('Alinea',
  'Sous la présidence de monsieur le maire suppléant Robert Lagacé. Tous membres du conseil et en formant le quorum.'),
 ('Alinea',
  'Monsieur le maire Robert Milot était absent pour toute la durée de la séance. ATTENDU la politique culturelle adoptée en septembre 2008 par le conseil municipal;'),
 ('Alinea',
  "ATTENDU QUE le conseil reconnait l’importance de la culture dans le développement de sa collectivité et plus encore, qu’il accepte la stratégie d'action portée par les citoyens et les organismes culturels adél